<a href="https://colab.research.google.com/github/InowaR/colab/blob/main/fastapi_postgres.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt install postgresql postgresql-contrib
!sudo service postgresql start
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'q';"
!sudo -u postgres psql -U postgres -c 'CREATE DATABASE fastapi_crud;'
!export DATABASE_URL=postgresql://postgres:q@localhost:5432/fastapi_crud

In [ ]:
!pip install fastapi daphne

In [12]:
from google.colab.output import eval_js
link = eval_js("google.colab.kernel.proxyPort(8000)")
print(f'{link}docs')

https://diij7sppxf-496ff2e9c6d22116-8000-colab.googleusercontent.com/docs


In [37]:
app = """from fastapi import FastAPI, Body, HTTPException
from pydantic import BaseModel
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker, declarative_base
from sqlalchemy import Table, Column, Integer, String, Float


app = FastAPI()

DATABASE_URL = "postgresql://postgres:q@localhost:5432/fastapi_crud"

engine = create_engine(DATABASE_URL)

Base = declarative_base()
Base.metadata.create_all(engine)

Session = sessionmaker(bind=engine)


class Product(BaseModel):
    name: str
    price: float


class ProductDB(Base):
    __tablename__ = 'products'

    id = Column(Integer, primary_key=True)
    name = Column(String(255), nullable=False)
    price = Column(Float, nullable=False)


@app.post("/products")
async def create_product(product: Product = Body(...)):
    db_product = ProductDB(**product.dict())
    with Session() as session:
        session.add(db_product)
        session.commit()
        return {"message": "Продукт успешно добавлен"}


@app.get("/products")
async def get_all_products():
    with Session() as session:
        products = session.query(ProductDB).all()
        return [product for product in products]


@app.put("/products/{pk}")
async def update_product_by_id(pk: int, product: Product = Body(...)):
    with Session() as session:
        db_product = session.query(ProductDB).filter(ProductDB.id == pk).first()
        if not db_product:
            raise HTTPException(status_code=404, detail="Продукт не найден")
        updated_product = ProductDB(**product.dict())
        db_product.name = updated_product.name
        db_product.price = updated_product.price
        session.commit()
        return {"message": "Продукт успешно обновлен"}


@app.delete("/products/{pk}")
async def delete_product_by_id(pk: int):
    with Session() as session:
        db_product = session.query(ProductDB).filter(ProductDB.id == pk).first()
        if not db_product:
            raise HTTPException(status_code=404, detail="Продукт не найден")
        session.delete(db_product)
        session.commit()
        return {"message": "Продукт успешно удален"}
"""

with open('main.py', 'w') as m:
    m.write(app)

In [42]:
!daphne -b 127.0.0.1 -p 8000 main:app

2024-04-19 09:01:02,389 INFO     Starting server at tcp:port=8000:interface=127.0.0.1
2024-04-19 09:01:02,390 INFO     HTTP/2 support not enabled (install the http2 and tls Twisted extras)
2024-04-19 09:01:02,390 INFO     Configuring endpoint tcp:port=8000:interface=127.0.0.1
2024-04-19 09:01:02,391 INFO     Listening on TCP address 127.0.0.1:8000
127.0.0.1:56608 - - [19/Apr/2024:09:01:56] "POST /products" 200 60
2024-04-19 09:02:00,857 INFO     Killed 0 pending application instances
^C


In [44]:
!sudo -u postgres psql -U postgres fastapi_crud -c 'SELECT * FROM products;'

 id |   name    | price  
----+-----------+--------
  2 | телевизор | 200.78
(1 row)

>8